# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Прочитаем файл, выведем информацию о нем и первые 10 строк таблицы.

In [1]:
import pandas as pd
bank = pd.read_csv('/datasets/data.csv')
bank.info()
bank.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

При изучении общей информации, были найдены пропуски в колонках с трудовым стажем и общим доходом, а также значения в этих колонках являются вещественными. В колонке с трудовым стажем имеются отрицательные значения и значения с невозможно большой велечиной(340266.072047), а в образовании значения находятся в разных регистрах.

## Шаг 2. Предобработка данных

### Обработка пропусков

Преобразуем отрицательные значения.

In [3]:
# Для начала преобразуем значения в колонке со стажем в положительные методом abs()
bank['days_employed'] = bank['days_employed'].abs()

# проверяем результат методом loc и count()
bank[bank['days_employed'] < 0]['days_employed'].count()

0

Посмотрим какие значения пропущенны

In [3]:
bank.loc[bank['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Значения пропущенны в одних строках и скорее всего вызваны ошибкой в записи или хранении данных. 
Чтоб не потерять важные для анализа данные будем использовать метод fillna().

Заменим пропуски в колонке days_employed.

In [4]:
# т.к. мы не можем воспользоватся средним значением из за выбросов, найдем медиану
employed_median = bank['days_employed'].median()

# заменим пропущенные значения медианой с помощью метода fillna()
bank['days_employed'] = bank['days_employed'].fillna(employed_median)

# проверим результат методом info()
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Найдем значение после которого начинаются выбросы.

In [5]:
# найдем максимально возможный стаж
max_employed = (bank['dob_years'].max() - 16) * 365
max_employed

21535

Заменим невозможные значения в колонке days_employed.

In [6]:
# заменим выбросы на медиану методом loc()
bank.loc[bank['days_employed'] > max_employed, 'days_employed'] = employed_median

# проверим методом count()       
bank[bank['days_employed'] > max_employed]['days_employed'].count()

0

Заменим пропуски в колонке total_income.

In [4]:
# найдем все виды занятости в пропущеных значениях
bank[bank['total_income'].isna()]['income_type'].value_counts()
# найдем среднее значение для каждого типа
# для этого напишем функцию
def mean_f(income_type):
    result = 0
    result = bank[bank['income_type'] == income_type]['total_income'].mean()
    return result
    
sotr_mean = mean_f('сотрудник')
comp_mean = mean_f('компаньон')
penc_mean = mean_f('пенсионер')
gos_mean = mean_f('госслужащий')
pred_mean = mean_f('предприниматель')

# заменим пропуски на среднее значение методом fillna()
# для этого напишем функцию
def fillna_f(income_type, mean):
    bank.loc[bank['income_type'] == income_type, 'total_income'] = bank[bank['income_type'] == income_type]['total_income'].fillna(mean)
    
fillna_f('сотрудник', sotr_mean)
fillna_f('компаньон', comp_mean)
fillna_f('пенсионер', penc_mean)
fillna_f('госслужащий', gos_mean)
fillna_f('предприниматель', pred_mean)

In [5]:
# проверим методом count()

bank[bank['total_income'].isna()]['income_type'].count()

0

**Вывод**

В данных имелись пропуски, которые скорее всего были вызваны технической ошибкой. Мы заполнили их методом fillna(), для того чтобы не потерять важные для иследования данные. Также мы произвели замену выбросов на медианное значение для более точного анализа.

### Замена типа данных

Заменим значения float на int

In [8]:
# поменяем тим данных методом astype()
bank['days_employed'] = bank['days_employed'].astype('int')
bank['total_income'] = bank['total_income'].astype('int')
# проверим результат
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

В колонках с трудовым стажем в днях и ежемесячным доходом были значения с вещественным типом, которые мы поменяли на цельночисленные. Для замены мы использовали метод astype() т.к. он позволяет изменить вещественный тип данных на цельночисленный.

### Обработка дубликатов

Удалим дубликаты категориальных значений в регистре колонки education, приведя их значения к нижнему регистру.

In [6]:
# приведем все значения к нижнему регистру методом str.lower()
bank['education'] = bank['education'].str.lower()
# проверим результат методом unique()
bank['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Попробуем найти "грубые" дубликаты.

In [7]:
# найдем кол-во дубликатов методами duplicated() и sum()
bank.duplicated().sum()

71

Удалим их.

In [8]:
# удалим дупликаты методом drop_duplicates() и удалим индексы методом reset_index()
bank = bank.drop_duplicates().reset_index(drop=True)
# проверим результат
bank.duplicated().sum()

0

**Вывод**

В данных были дубликаты двух видов - "грубые" и с разым регистром. "Грубые" дубликаты это - следствие технической ошибки, а дубликаты с разным регистром - это ошибки при заполнении данных. Для поиска "грубых" дубликатов мы использовали метод duplicated(), а для удаления метод drop_duplicates с reset_index для удаления индексов. А что бы удалить дубликаты в регистре мы использовали метод str.lower() для приведения значений к нижнему регистру.

### Лемматизация

Импортируем библиотеку с функцией лемматизации.

In [12]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Выведем из целей для кредита уникальные значения и изучим информацию.

In [13]:
bank['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Мы видим, что основные цели для кредита это различные операции с недвижимостью, автомобилем, свадьбой и образованием.
Выведем леммы для данных значений.

In [14]:
lemm = []
for row in bank['purpose']:
    lemm += m.lemmatize(row)
Counter(lemm)


Counter({'покупка': 5900,
         ' ': 33596,
         'жилье': 4461,
         '\n': 21471,
         'приобретение': 461,
         'автомобиль': 4308,
         'дополнительный': 907,
         'образование': 4014,
         'сыграть': 769,
         'свадьба': 2335,
         'операция': 2604,
         'с': 2918,
         'на': 2228,
         'проведение': 773,
         'для': 1290,
         'семья': 638,
         'недвижимость': 6353,
         'коммерческий': 1312,
         'жилой': 1231,
         'строительство': 1879,
         'собственный': 635,
         'подержать': 478,
         'свой': 2231,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1315,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 652,
         'ремонт': 607})

Выпишем основыне леммы в отдельную строку и проведем для них стемминг.

In [15]:
lemm_purpose = ['жилье', 'недвижимость', 'автомобиль', 'свадьба', 'образование']

In [16]:
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')

for query in lemm_purpose:
    for word in query.split():
        stemmed_word = russian_stemmer.stem(word)
        print(stemmed_word)

жил
недвижим
автомобил
свадьб
образован


**Вывод**

Мы изучили данные по целям для получения кредита и провели для них лемматизацию, после чего выделили основные леммы с помощью библиотеки pymystemm и применили к ним стемминг с помощью библиотеки NLTK, подготовив данные для дальнейшего анализа.

### Категоризация данных

Для дальнейшей работы проверим данные на наличие дополнительных ошибок.

In [17]:
# проверим корректность данных в колонке children
bank['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мы видим что данная колонка имеет не корректные данные в небольшом обьеме, поэтому мы их удалим

In [9]:
# предположим что значения -1 и 20 возникли при некорректной записи
# заменим из на значения 2 и 1
bank['children'] = bank['children'].replace(20, 2)
bank['children'] = bank['children'].replace(-1, 1)

# проверим результат
bank['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Проверим колонку с возрастом клиента

In [10]:
print(bank['dob_years'].min())
print(bank['dob_years'].max())

0
75


Мы видим что имеются некоректные значения. Проверим сколько значений меньше 18 лет

In [11]:
bank[bank['dob_years'] < 18]['dob_years'].count()

101

Т.к. значений немного мы можем их удалить

In [12]:
bank = bank[bank['dob_years'] >= 18]
bank[bank['dob_years'] < 18]['dob_years'].count()

0

Проверим совпадает ли кол-во значений и их идентификаторов в колонках с уровнем образования и семйным положением

In [22]:
print(len(bank['education'].unique()))
print(len(bank['education_id'].unique()))

5
5


In [23]:
print(len(bank['family_status'].unique()))
print(len(bank['family_status_id'].unique()))

5
5


Проверим колонку с полом клиента

In [24]:
bank['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [25]:
bank['gender'].value_counts()

F      14036
M       7211
XNA        1
Name: gender, dtype: int64

Имеется всего 1 некоректное значение от которого мы можем избавится

In [26]:
bank = bank[bank['gender'] != 'XNA']
bank['gender'].value_counts()

F    14036
M     7211
Name: gender, dtype: int64

Проверим колонку с типом занятости клиента

In [27]:
bank['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

И проверим колонку с задолжностью по возвратам

In [28]:
bank['debt'].unique()

array([0, 1])

Теперь данные очищены и готовы к дальнейшей работе. Категоризируем их по целям получения кредита.

In [50]:
# создадим функцию для разделения целей на категории
def purpose_lemmas(row):
    pur = row['purpose']
    if ('недвижим' in pur) or ('жил' in pur):
        return 'Недвижимость'
    elif 'автомобил' in pur:
        return 'Автомобиль'
    elif 'свадьб' in pur:
        return 'Свадьба'
    elif 'образован' in pur:
        return 'Образование'
    else:
        return 'Неизвестно'


In [51]:
# применим функцию и проверим результат
bank['purpose_category'] = bank.apply(purpose_lemmas, axis=1)
bank.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,Выше среднего,Недвижимость
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,Ниже среднего,Автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,Средний,Недвижимость
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,Выше среднего,Образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,Средний,Свадьба
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,Выше среднего,Недвижимость
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,Выше среднего,Недвижимость
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,Средний,Образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,Ниже среднего,Свадьба
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,Средний,Недвижимость


In [52]:
# проверим есть ли неизвестные категории
bank[bank['purpose_category'] == 'Неизвестно']['purpose_category'].count()

0

Категоризируем данные по ежемесячному доходу

In [41]:
# найдем квантили
t_mean = bank['total_income'].mean
bank['total_income'].quantile([0.33, 0.66])

0.33    122201.772628
0.66    174860.931712
Name: total_income, dtype: float64

In [42]:
# запишем квантили в переменные
q33 = 122201.772628
q66 = 174860.931712

In [43]:
# напишем функцию для деления по уровню дохода
def income_category(row):
    inc = row['total_income']
    if inc < q33:
        return 'Ниже среднего'
    elif q33 < inc < q66:
        return 'Средний'
    elif inc > q66:
        return 'Выше среднего'

In [44]:
# применим функцию и проверим результат
bank['income_category'] = bank.apply(income_category, axis=1)
bank.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,Выше среднего
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,Ниже среднего
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,Средний
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,Выше среднего
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,Средний
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,Выше среднего
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,Выше среднего
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,Средний
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,Ниже среднего
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,Средний


**Вывод**

Мы очистили данные от оставшихся ошибок и категоризировали их. Добавив категории с целью кредита и уровнем дохода мы сможем точнее ответить на поставленные вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
children_pivot = bank.pivot_table(index=['children'], columns='debt', values='gender', aggfunc='count')
children_pivot[1] = children_pivot[1].fillna(0)
children_pivot['%'] = children_pivot[1] / (children_pivot[0] + children_pivot[1]) * 100
children_pivot

debt,0,1,%
children,,,
0,12964.0,1058.0,7.545286
1,4397.0,442.0,9.134119
2,1912.0,202.0,9.555345
3,301.0,27.0,8.231707
4,37.0,4.0,9.756098
5,9.0,0.0,0.000000


**Вывод**

Людей с детьми почти в два раза меньше, а должников среди них больше. За исключением семей с 5ю детьми, у них 100% кредитов выплаченны.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
merried_pivot = bank.pivot_table(index=['family_status'], columns='debt', values='gender', aggfunc='count')
merried_pivot['%'] = merried_pivot[1] / (merried_pivot[0] + merried_pivot[1]) * 100
merried_pivot

debt,0,1,%
family_status,,,
Не женат / не замужем,2521,273,9.770938
в разводе,1100,85,7.172996
вдовец / вдова,892,62,6.498952
гражданский брак,3744,386,9.346247
женат / замужем,11363,927,7.542718


**Вывод**

Меньше всего должников оказалось в категории вдовец / вдова, далее идут клиенты в разводе и в браке. Хуже всех выплачивают кредиты клиенты в гражданском браке и те, кто не женат / не замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
income_pivot = bank.pivot_table(index=['income_category'], columns='debt', values='gender', aggfunc='count')
income_pivot['%'] = income_pivot[1] / (income_pivot[0] + income_pivot[1]) * 100
income_pivot

debt,0,1,%
income_category,,,
Выше среднего,6723,537,7.396694
Ниже среднего,6471,576,8.173691
Средний,6426,620,8.799319


**Вывод**

В результате исследования выяснилось, что среди клиентов с доходом выше среднего должников меньше всего. Далее идут клиенты с доходом ниже среднего, и на последнем месте оказались клиенты со средним доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [54]:
purpose_pivot = bank.pivot_table(index=['purpose_category'], columns='debt', values='gender', aggfunc='count')
purpose_pivot['%'] = purpose_pivot[1] / (purpose_pivot[0] + purpose_pivot[1]) * 100
purpose_pivot

debt,0,1,%
purpose_category,,,
Автомобиль,3884,400,9.337068
Недвижимость,9985,779,7.237087
Образование,3625,370,9.261577
Свадьба,2126,184,7.965368


**Вывод**

Среди тех кто берет кредит на недвижимость и свадьбу должников меньше чем среди тех кто берет кредит на автомобиль или образование.

## Шаг 4. Общий вывод

По результатам исследования можно сделать вывод, что наиболее платежеспособные клиенты - это клиенты без детей, клиенты относящиеся к категории вдовец\вдова, кленты с доходом выше среднего и те, у кого целью кредита является недвижимость или свадьба.